In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.metrics import classification_report

In [2]:
#Data import
train_df = pd.read_csv("./Lightning7/Lighting7_TRAIN.txt", header=None)
test_df = pd.read_csv("./Lightning7/Lighting7_TEST.txt",header=None)
all_data = pd.concat([train_df, test_df]) #combining data to experiment with different train/test splits

In [19]:
classes = {i:all_data.loc[all_data[0]==i] for i in range(7)}

In [47]:
def euclidean_dist(t1, t2):
    return math.sqrt(sum((t1-t2)**2))



In [65]:
def DTWDistance(s1, s2,w):
    DTW={}
    
    w = max(w, abs(len(s1)-len(s2)))
    
    for i in range(-1,len(s1)):
        for j in range(-1,len(s2)):
            DTW[(i, j)] = float('inf')
    DTW[(-1, -1)] = 0
  
    for i in range(len(s1)):
        for j in range(max(0, i-w), min(len(s2), i+w)):
            dist= (s1[i]-s2[j])**2
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])
    return math.sqrt(DTW[len(s1)-1, len(s2)-1])

In [77]:
def LB_Keogh(s1,s2,r):
    LB_sum=0
    for ind,i in enumerate(s1):
        
        lower_bound=min(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
        upper_bound=max(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
        
        if i>upper_bound:
            LB_sum=LB_sum+(i-upper_bound)**2
        elif i<lower_bound:
            LB_sum=LB_sum+(i-lower_bound)**2
    
    return math.sqrt(LB_sum)

In [78]:
def knn(train,test,w):
    preds=[]
    for ind,i in enumerate(test):
        min_dist=float('inf')
        closest_seq=[]
        #print ind
        for j in train:
            if LB_Keogh(i[1:],j[1:],5)<min_dist:
                dist=DTWDistance(i[1:],j[1:],w)
                if dist<min_dist:
                    min_dist=dist
                    closest_seq=j
        preds.append(closest_seq[0])
    return classification_report(test[:,0],preds)

In [80]:
print (knn(train_df.values, test_df.values, 4))

             precision    recall  f1-score   support

        0.0       0.77      1.00      0.87        10
        1.0       0.00      0.00      0.00         9
        2.0       0.50      0.50      0.50         6
        3.0       0.46      0.86      0.60         7
        4.0       1.00      0.20      0.33        10
        5.0       0.67      0.84      0.74        19
        6.0       0.92      0.92      0.92        12

avg / total       0.65      0.66      0.61        73

